- Họ tên: Võ Minh Tùng
- MSHV : 2370345

## Vision tranformer

install stanford card dataset keras

In [1]:
!pip install timm






[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


Chuẩn bị Bộ Dữ Liệu Stanford Cars

In [6]:
import tensorflow as tf
import timm
from tensorflow.keras.preprocessing import image_dataset_from_directory

# Path to the Stanford Cars dataset
data_dir = "./stanford_cars"

# Load training and validation data
batch_size = 32
img_size = (224, 224)

train_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=img_size,  # Ensures images are resized to 224x224
    batch_size=batch_size,
)

val_ds = image_dataset_from_directory(
    data_dir,
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=img_size,  # Ensures images are resized to 224x224
    batch_size=batch_size,
)

# Define preprocessing function to normalize images
def preprocess_image(image, label):
    image = tf.image.resize(image, img_size)  # Resize to 224x224
    image = tf.cast(image, tf.float32) / 255.0  # Normalize to [0, 1]
    return image, label

# Apply preprocessing to both datasets
train_ds = train_ds.map(preprocess_image)
val_ds = val_ds.map(preprocess_image)


Found 16185 files belonging to 2 classes.
Using 12948 files for training.
Found 16185 files belonging to 2 classes.
Using 3237 files for validation.


Step 3: Prepare a Custom ViT Model Using timm

In [10]:
import numpy as np
from tensorflow.keras import layers, models

# Load a pre-trained ViT model from timm
vit_model = timm.create_model('vit_base_patch16_224', pretrained=True, num_classes=num_classes)

# Wrap the model with the TensorFlow model class
class ViTTensorFlow(tf.keras.Model):
    def __init__(self, vit_model):
        super(ViTTensorFlow, self).__init__()
        self.vit_model = vit_model

    def call(self, inputs):
        return self.vit_model(inputs)

# Initialize the model with input shape matching 224x224x3
model = ViTTensorFlow(vit_model)
model.build(input_shape=(None, *img_size, 3))


Step 4: Compile the Model

In [12]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=["accuracy"]
)

Step 5: Train the Model

In [15]:

# Train the model
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=5  # Adjust as needed
)



Epoch 1/5


AssertionError: Exception encountered when calling ViTTensorFlow.call().

[1mInput width (3) doesn't match model (224).[0m

Arguments received by ViTTensorFlow.call():
  • inputs=tf.Tensor(shape=(None, 224, 224, 3), dtype=float32)

In [43]:
from transformers import TFViTForImageClassification


# Tải mô hình Vision Transformer
model = TFViTForImageClassification.from_pretrained('google/vit-base-patch16-224')

# Biên dịch mô hình
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4),
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

# Hiển thị tóm tắt mô hình
model.summary()


RuntimeError: Failed to import transformers.models.vit.modeling_tf_vit because of the following error (look up to see its traceback):
Your currently installed version of Keras is Keras 3, but this is not yet supported in Transformers. Please install the backwards-compatible tf-keras package with `pip install tf-keras`.